In [ ]:
from astropy.coordinates import SkyCoord
from astropy.io import fits
import healpy as hp
import matplotlib.pyplot as plt
import time
import sys
from pixell import enmap, enplot, reproject, utils, curvedsky, wcsutils
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.linalg import sqrtm
from tqdm import tqdm
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import numpy as np
import pandas as pd

maps = ["../data/act_dr5.01_s08s18_AA_f090_daynight_map.fits",
        "../data/act_dr5.01_s08s18_AA_f150_daynight_map.fits",
        "../data/act_dr5.01_s08s18_AA_f220_daynight_map.fits"]

for x in range(len(maps)):
    beam = 1.4
    filepath = maps[x]
    
    map_pix = enmap.read_map(filepath)
    return map_pix

    awd_not_adv = pd.read_csv('../data/awd_not_adv.csv', skiprows = 0)

    zs = [0, 0.32, 100]
    rhs = [0, 27, 100000]

    stacks = []
    lens = []
    riches = []
    reds = []

    for j in range(len(zs)):
        for k in range(len(rhs)):
            stack_CMB_kSZ           = 0
            stack_CMB_kSZ_deproject = 0
            c = 0
            riches.append(rhs[k])
            reds.append(zs[j])
            if j == len(zs)-1:
                if k == len(rhs)-1:
                    ras = awd_not_adv[(awd_not_adv.amf_z<zs[j]) & (awd_not_adv.amf_z>0) & (awd_not_adv.amf_rh<rhs[k]) & (awd_not_adv.amf_rh>0)]['amf_ra'].reset_index(drop=True)
                    decs = awd_not_adv[(awd_not_adv.amf_z<zs[j]) & (awd_not_adv.amf_z>0) & (awd_not_adv.amf_rh<rhs[k]) & (awd_not_adv.amf_rh>0)]['amf_dec'].reset_index(drop=True)
                else:
                    ras = awd_not_adv[(awd_not_adv.amf_z<zs[j]) & (awd_not_adv.amf_z>0) & (awd_not_adv.amf_rh<rhs[k+1]) & (awd_not_adv.amf_rh>rhs[k])]['amf_ra'].reset_index(drop=True)
                    decs = awd_not_adv[(awd_not_adv.amf_z<zs[j]) & (awd_not_adv.amf_z>0) & (awd_not_adv.amf_rh<rhs[k+1]) & (awd_not_adv.amf_rh>rhs[k])]['amf_dec'].reset_index(drop=True)
            elif k == len(rhs)-1:
                ras = awd_not_adv[(awd_not_adv.amf_z<zs[j+1]) & (awd_not_adv.amf_z>zs[j]) & (awd_not_adv.amf_rh<rhs[k]) & (awd_not_adv.amf_rh>0)]['amf_ra'].reset_index(drop=True)
                decs = awd_not_adv[(awd_not_adv.amf_z<zs[j+1]) & (awd_not_adv.amf_z>zs[j]) & (awd_not_adv.amf_rh<rhs[k]) & (awd_not_adv.amf_rh>0)]['amf_dec'].reset_index(drop=True)
            else:
                ras = awd_not_adv[(awd_not_adv.amf_z<zs[j+1]) & (awd_not_adv.amf_z>zs[j]) & (awd_not_adv.amf_rh<rhs[k+1]) & (awd_not_adv.amf_rh>rhs[k])]['amf_ra'].reset_index(drop=True)
                decs = awd_not_adv[(awd_not_adv.amf_z<zs[j+1]) & (awd_not_adv.amf_z>zs[j]) & (awd_not_adv.amf_rh<rhs[k+1]) & (awd_not_adv.amf_rh>rhs[k])]['amf_dec'].reset_index(drop=True)

            N = len(ras)
            lens.append(N)

            for i in range(N):
                stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=beam*utils.arcmin)

                if stamp is None:
                    continue
                elif stamp[0][0][0]==0.0:
                    continue
                else:
                    stack_CMB_kSZ += np.array(stamp)
                    c+=1

            # Normalize the stamps
            stack_CMB_kSZ /= c
            stack_CMB_kSZ_deproject /= c

            stacks.append(stack_CMB_kSZ[0])

    f, ax = plt.subplots(nrows=3, ncols=3,figsize=(15,15))
    i = 0

    for j,row in enumerate(ax):
        for k, col in enumerate(row):
            f1 = col.imshow(stacks[i])
            col.get_xaxis().set_visible(False)
            col.get_yaxis().set_visible(False)
            plt.colorbar(mappable = f1, ax = col,orientation='horizontal',fraction=0.047, pad=0.02)

            col.text(3,4,"N={}".format(lens[i]),color='red')
            if j==0 and k==0:
                col.text(-15,20,"z<0.32",size=15)
            if j==1 and k==0:
                col.text(-15,20,"z>0.32",size=15)
            if j==2 and k==0:
                col.text(-15,20,"all z",size=15)
            if k==0 and j==0:
                col.text(15,-5,"rh<27",size=15)
            if k==1 and j==0:
                col.text(15,-5,"rh>27",size=15)
            if k==2 and j==0:
                col.text(15,-5,"all rh",size=15)

            i+=1

    plt.savefig("rosat_{}.png".format(x))

In [4]:
import numpy as np
arr = np.array([[1,2],[3,4]])

In [5]:
arr-1

array([[0, 1],
       [2, 3]])